In [1]:
import os
from dotenv import load_dotenv
from google.cloud import bigquery
import pandas as pd

In [2]:
# --- Carga das variáveis de ambiente do arquivo .env ---
load_dotenv()

True

In [3]:
# --- Configuração da autenticação com Google Cloud ---
current_dir = os.getcwd()
json_path = os.path.abspath(os.path.join(current_dir, '..', 'credenciais', 'bq-chave.json'))

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = json_path

# --- Inicializa cliente BigQuery ---
client = bigquery.Client()

# --- Definições padrão do projeto, dataset e tabela ---
project_id = 'telemedicina-462920'
dataset_id = 'saude'

# Função para construir o full_table_id no formato 'projeto.dataset.tabela'
def get_full_table_id(project, dataset, table):
    return f"{project}.{dataset}.{table}"

In [4]:
# --- Função genérica para carregar dataframe Excel e enviar para BigQuery ---
def upload_to_bigquery_from_excel(sheet_name: str, table_name: str, schema: list):
    excel_path = os.getenv("EXCEL_FILE_PATH")
    if not excel_path or not os.path.exists(excel_path):
        raise FileNotFoundError("Arquivo Excel não encontrado ou variável EXCEL_FILE_PATH não definida.")
        
    df = pd.read_excel(excel_path, sheet_name=sheet_name)
    
    # Converte colunas do tipo DATE para datetime no pandas
    date_columns = [field.name for field in schema if field.field_type == "DATE"]
    for col in date_columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
    
    # Não precisa ler novamente aqui
    print(f"\nDados carregados da aba '{sheet_name}':")
    print(f"Tamanho (linhas, colunas): {df.shape}")
    print(f"Tipos de dados:\n{df.dtypes}")
    
    job_config = bigquery.LoadJobConfig(
        schema=schema,
        write_disposition="WRITE_TRUNCATE"
    )
    
    full_table_id = get_full_table_id(project_id, dataset_id, table_name)
    
    job = client.load_table_from_dataframe(df, full_table_id, job_config=job_config)
    job.result()
    
    print(f"✅ Dados enviados com sucesso para: {full_table_id}")

In [5]:
# --- Exemplo de uso da função para a aba e tabela meta_odonto ---
schema_saude = [
    bigquery.SchemaField("Data contratação", "DATE"),
    bigquery.SchemaField("Data", "DATE"),
    bigquery.SchemaField("Produto", "STRING"),
    bigquery.SchemaField("Estado", "STRING"),
    bigquery.SchemaField("Nome", "STRING"),
    bigquery.SchemaField("Operadora", "STRING"),
    bigquery.SchemaField("Origem", "STRING"),
    bigquery.SchemaField("Status", "STRING"),
    bigquery.SchemaField("Vendedor", "STRING"),
    bigquery.SchemaField("Valor", "FLOAT"),
]

upload_to_bigquery_from_excel(
    sheet_name='saúde', 
    table_name='saude', 
    schema=schema_saude
)


Dados carregados da aba 'saúde':
Tamanho (linhas, colunas): (468, 10)
Tipos de dados:
Data contratação    datetime64[ns]
Data                datetime64[ns]
Produto                     object
Estado                      object
Nome                        object
Operadora                   object
Origem                      object
Status                      object
Vendedor                    object
Valor                      float64
dtype: object


C:\Anaconda\lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:489: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


✅ Dados enviados com sucesso para: telemedicina-462920.saude.saude
